<a href="https://colab.research.google.com/github/choi-yh/DataMining/blob/master/8_3_rnn_longchar_many2many_TF2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 좀 더 긴 문장을 학습. Batch와 Stacked RNN이 필요하다.
* 많은 문장을 학습할 때, 글 전체를 하나의 Sequence로 학습하는 것은 적절하지 않다.
* 하나의 문장은 나름 연관된 구조를 가지고 있지만, 글 전체로 보면 앞 문장과 뒤 문장의 견관성이 떨어진다.
* 이 때, Batch 학습을 한다.
* Batch 학습은 독립적으로 인정되는 단위로 끊어서 학습하는 방법
* 만약, 10자 씩 끊어서 학습한다면 "That poor", "hatm poor", "at, poor c"와 같이 여러 개의 문장이 나올 것이다.
* 하나의 문장을 학습하다가 Batch 학습을 하면 모형의 정확도가 급속도로 떨어진다.
* 모형의 정확도를 올리기 위해서는 Cell의 수를 크게 해야 한다.
* 더 정확도를 올리려면 Recurrent 층을 여러 개로 확장하는 Stacked RNN을 사용해야 한다.

In [ ]:
import numpy as np
import urllib.request
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [ ]:
lines = ["if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea."]

In [ ]:
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 180


In [ ]:
print(text)

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


In [ ]:
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)
print('글자 집합의 크기: {}'.format(vocab_size))

글자 집합의 크기: 25


In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'w': 23, 'y': 24}


숫자 인덱스를 받아서 문자로 바꾸는 딕셔너리를 만들고 문장의 10개 단위로 끊어 첫 문장이 다음 문장을 예측하도록 train_X와 train_y를 생성했다.

In [ ]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [ ]:
seq_length = 10 # 문장의 길이는 10으로 한다.
n_samples = int(np.floor((len(text) - 1) / seq_length)) # 문자열을 10등분 한다. 그러면 즉, 총 샘플의 개수
print('문장 샘플의 수: {}'.format(n_samples))

문장 샘플의 수: 17


In [ ]:
_tmp = []

for i in range(n_samples):
    X_sample = text[i * seq_length: (i+1) * seq_length]
    # 0:10 -> 10:20 -> 20:30로 loop 돌면서 문장 샘플을 1개씩 가져온다.
    X_encoded = [char_to_index[c] for c in X_sample] # 하나의 문장 샘플에 대해서 정수 인코딩
    _tmp.append(X_encoded)

In [ ]:
train_X = _tmp[:-1]
train_y = _tmp[1:]

In [ ]:
print(train_X)
print(train_y)

[[12, 9, 0, 24, 17, 22, 0, 23, 4, 16], [21, 0, 21, 17, 0, 5, 22, 12, 14, 7], [0, 4, 0, 20, 11, 12, 18, 2, 0, 7], [17, 16, 1, 21, 0, 7, 19, 22, 15, 0], [22, 18, 0, 18, 8, 17, 18, 14, 8, 0], [21, 17, 10, 8, 21, 11, 8, 19, 0, 21], [17, 0, 6, 17, 14, 14, 8, 6, 21, 0], [23, 17, 17, 7, 0, 4, 16, 7, 0, 7], [17, 16, 1, 21, 0, 4, 20, 20, 12, 10], [16, 0, 21, 11, 8, 15, 0, 21, 4, 20], [13, 20, 0, 4, 16, 7, 0, 23, 17, 19], [13, 2, 0, 5, 22, 21, 0, 19, 4, 21], [11, 8, 19, 0, 21, 8, 4, 6, 11, 0], [21, 11, 8, 15, 0, 21, 17, 0, 14, 17], [16, 10, 0, 9, 17, 19, 0, 21, 11, 8], [0, 8, 16, 7, 14, 8, 20, 20, 0, 12]]
[[21, 0, 21, 17, 0, 5, 22, 12, 14, 7], [0, 4, 0, 20, 11, 12, 18, 2, 0, 7], [17, 16, 1, 21, 0, 7, 19, 22, 15, 0], [22, 18, 0, 18, 8, 17, 18, 14, 8, 0], [21, 17, 10, 8, 21, 11, 8, 19, 0, 21], [17, 0, 6, 17, 14, 14, 8, 6, 21, 0], [23, 17, 17, 7, 0, 4, 16, 7, 0, 7], [17, 16, 1, 21, 0, 4, 20, 20, 12, 10], [16, 0, 21, 11, 8, 15, 0, 21, 4, 20], [13, 20, 0, 4, 16, 7, 0, 23, 17, 19], [13, 2, 0, 5, 22, 2

train_X 와 train_y 를 one-hot encoding 하고 3단 Stacked RNN 모형을 생성  
return_sequence=True 와 TimeDistributed 함수를 이용해 출력이 many2many 형태로 계산하게 하였다.  
many2many의 경우, cost는 전 Sequence에 대해 평균해 계산한다.

In [ ]:
encoded_X = to_categorical(train_X)
encoded_y = to_categorical(train_y)

In [ ]:
# one-hot encoding
print('encoded_X의 크기(shape): {}'.format(encoded_X.shape))
print('encoded_y의 크기(shape: {}'.format(encoded_y.shape))

encoded_X의 크기(shape): (16, 10, 25)
encoded_y의 크기(shape: (16, 10, 25)


In [ ]:
n_hidden = 50
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(None, encoded_X.shape[2]), return_sequences=True))
model.add(LSTM(n_hidden, return_sequences=True))
model.add(LSTM(n_hidden, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(encoded_X, encoded_y, epochs=1000, verbose=2)

Epoch 1/1000
1/1 - 0s - loss: 3.2190 - accuracy: 0.0000e+00
Epoch 2/1000
1/1 - 0s - loss: 3.2153 - accuracy: 0.1500
Epoch 3/1000
1/1 - 0s - loss: 3.2115 - accuracy: 0.1875
Epoch 4/1000
1/1 - 0s - loss: 3.2075 - accuracy: 0.1875
Epoch 5/1000
1/1 - 0s - loss: 3.2031 - accuracy: 0.1875
Epoch 6/1000
1/1 - 0s - loss: 3.1984 - accuracy: 0.1875
Epoch 7/1000
1/1 - 0s - loss: 3.1932 - accuracy: 0.1875
Epoch 8/1000
1/1 - 0s - loss: 3.1873 - accuracy: 0.1875
Epoch 9/1000
1/1 - 0s - loss: 3.1806 - accuracy: 0.1875
Epoch 10/1000
1/1 - 0s - loss: 3.1730 - accuracy: 0.1875
Epoch 11/1000
1/1 - 0s - loss: 3.1643 - accuracy: 0.1875
Epoch 12/1000
1/1 - 0s - loss: 3.1542 - accuracy: 0.1875
Epoch 13/1000
1/1 - 0s - loss: 3.1425 - accuracy: 0.1875
Epoch 14/1000
1/1 - 0s - loss: 3.1290 - accuracy: 0.1875
Epoch 15/1000
1/1 - 0s - loss: 3.1135 - accuracy: 0.1875
Epoch 16/1000
1/1 - 0s - loss: 3.0960 - accuracy: 0.1875
Epoch 17/1000
1/1 - 0s - loss: 3.0767 - accuracy: 0.1875
Epoch 18/1000
1/1 - 0s - loss: 3.056

In [ ]:
print(text)

for vec in np.array([train_X[0]]):
    for idx in vec:
        print(index_to_char[idx], end="")

for i in range(16):
    pred = model.predict(encoded_X[i].reshape(1, 10, 25))
    pred = np.argmax(pred, 2)
    for vec in pred:
        for idx in vec:
            print(index_to_char[idx], end='')

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
if you want to buildoa ship, don't drum w  heople together to collect wood and don't assigw  hem tas s and work, but ratker teach them to loog for the endless iomensity o